### Importing additional relevant libraries for Part II

In [ ]:
## Scikit-learn built-in dataset generators
from sklearn.datasets import make_moons, make_circles, make_blobs

## Progress bar
import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam
from torch.distributions.multivariate_normal import MultivariateNormal

# Traning will be done on CPU for this homework. 
# For K=4, N=1500, epochs=1000 takes < 3 mins.
device = torch.device("cpu")
print("Using device", device)

## Part II: Invertible Neural Networks (70 points) 

In this part, we will take a closer look at invertible neural networks, otherwise known as *Normalizing Flows*. The most popular, current application of normalizing flows is to model datasets of images. In this part, we will implement a type of flows called *Coupling Flows* for a simplified problem of sampling from toy 2D datasets, although similar concepts (with deeper models + tricks) can be used to model images.

### General Concept 
Recall that given a random vector $Z$ with a density $p_Z(\mathbf{z})$ (e.g. Gaussian) and an invertible function $f$, the density $p_X(\mathbf{x})$ of $X=f(Z)$ is given by:

$$
\log p_X(\mathbf{x}) = \log p_Z(f^{-1}(\mathbf{x})) + \log{} \left|\det \frac{df^{-1}(\mathbf{x})}{d\mathbf{x}}\right|
$$

### Toy Datasets 

The provided function `sample_2d_datasets` samples from 4 different toy datasets that we will use for this part to experiment with NFs. The supported options in this function are `{'Circles', 'Moons', 'GaussiansGrid', 'GaussiansRot'}`, where for the last distribution `'GaussiansRot'`, the function supports a varying number of gaussians using the parameter `num_gaussians`.

In [ ]:
def sample_2d_datasets(dist_type, num_samples=1000, seed=0, num_gaussians=5):
  """
  function samples from simple pre-defined distributions in 2D.
  Inputs:
    - dist_type: str specifying the distribution to be chosen from:
      {'Circles', 'Moons', 'GaussiansGrid', 'GaussiansRot'}
    - num_samples: Number of samples to draw from dist_type (int).
    - seed: Random seed integer.
    - num_gaussians: Number of rotated gaussians if dist_type='GaussiansRot'. 
      (relevant only for dist_type='GaussiansRot', should be a keyword argument)
  Outputs:
    - data (np.array): array of num_samplesx2 samples from dist_type
  """
  np.random.seed(seed)
  if dist_type == 'Circles':
    data = make_circles(num_samples, noise=.1, factor=.8, random_state=seed, shuffle=True)[0]
  elif dist_type == 'Moons':
    data = make_moons(num_samples, noise=.1, random_state=seed, shuffle=True)[0]
  elif dist_type == 'GaussiansGrid':
    centers = np.array([[0,0],[0,2],[2,0],[2,2]])
    data = make_blobs(num_samples, centers=centers, cluster_std=.5, random_state=seed, shuffle=True)[0]
  elif dist_type == 'GaussiansRot':
    angles = np.linspace(0, 2 * np.pi, num_gaussians, endpoint=False)
    centers = np.stack([2.5 * np.array([np.cos(angle), np.sin(angle)]) for angle in angles])
    data = make_blobs(num_samples, centers=centers, cluster_std=np.sqrt(.1), random_state=seed, shuffle=True)[0]
  else:
    raise NotImplementedError
  return data

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 1</span>**. To get acquainted with this function, for each of the 4 distributions above, draw $N = 1000$ samples ${{x}_i}$. Display the drawn samples for each distribution in a separate plot.


In [ ]:
dists = ['Circles', 'Moons', 'GaussiansGrid', 'GaussiansRot']
samples = [sample_2d_datasets(dist_type) for dist_type in dists]

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,4,figsize=(15,3))
for i,sample in enumerate(samples):
    ax[i].scatter(sample[:,0],sample[:,1],s=3)
    ax[i].set_title(dists[i])
    plt.savefig('Toy Data Samples.png')

For convience purposes, we wrap the function `sample_2d_datasets` with a `torch.utils.data.Dataset` class, and implement the methods `__len__` and `__getitem__` to sample batches afterward with dataloaders when we train our models.

In [ ]:
class ToyDataset(Dataset):
  def __init__(self, dist_type, num_samples=1000, seed=0, num_gaussians=5):
      """
      Wrapper around the function "sample_2d_datasets" to allow iterating 
      batches using a datalaoder when training our normalizing flow model.
      """
      self.data = sample_2d_datasets(dist_type, num_samples, seed, num_gaussians)
      self.num_samples = num_samples

  def __len__(self):
      return self.num_samples

  def __getitem__(self, index):
      return torch.from_numpy(self.data[index]).type(torch.FloatTensor)

### Coupling Layers 

Next, we look at possible transformations to apply inside the flow, focusing on the simplest and most efficient one. A recent popular flow layer, which works well in combination with deep neural networks, is the coupling layer introduced by [Dinh et al.](https://arxiv.org/abs/1605.08803). The input $\mathbf{x}$ is arbitrarily split into two parts, $\mathbf{x}_{1:j}$ and $\mathbf{x}_{j+1:d}$, of which the first remains unchanged by the flow. Yet, $\mathbf{x}_{1:j}$ is used to parameterize the transformation for the second part, $\mathbf{x}_{j+1:d}$. In this coupling layer, we apply an affine transformation by scaling the input by $\mathbf{s}$ and shifting it by $\mathbf{t}$. In other words, our transformation $\mathbf{y}=f(\mathbf{x})$ looks as follows:

$$\mathbf{y}_{1:j} = \mathbf{x}_{1:j}$$
$$\mathbf{y}_{j+1:d} = \mathbf{s}_{\theta_1}(\mathbf{x}_{1:j}) \odot \mathbf{x}_{j+1:d} + \mathbf{t}_{\theta_2}(\mathbf{x}_{1:j})$$

$\mathbf{y}$ is the output of the flow layer, the functions $\mathbf{s}$ and $\mathbf{t}$ are implemented as neural networks, and the sum and multiplication are performed element-wise. Here's a block diagram that visualize the coupling layer in the form of a computation graph:

<center width="100%"><img src="https://drive.google.com/uc?id=1R_omjSvfBN4xjkJgkmkwPX10jkjz_mmC" width="400px"></center>

For convience, since we train using the log-density, it is common practice to apply an exponential on the predicted scaling factor prior to multiplication with $\mathbf{x}_{j+1:d}$, as this simplifies the calculation of the log-determinant of the Jacobian that we will derive shortly. Hence, the implemented transformation in practice is usually:

$$\mathbf{y}_{1:j} = \mathbf{x}_{1:j}$$
$$\mathbf{y}_{j+1:d} = \exp(\mathbf{s}_{\theta_1}(\mathbf{x}_{1:j})) \odot \mathbf{x}_{j+1:d} + \mathbf{t}_{\theta_2}(\mathbf{x}_{1:j})$$

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 2</span>**. Write down the inverse of this layer $\mathbf{x}=f^{-1}(\mathbf{y})$ for some $\mathbf{x},\mathbf{y} \in \mathbb{R}^d$. Draw the inverse function $f^{-1}(\mathbf{y})$ as a computational graph that has $\mathbf{y}$ as input and $\mathbf{x}$ as output.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 3</span>**. Write down the Jacobian of this layer $\frac{d \mathbf{y}}{d \mathbf{x}}$. Do you recognize a special structure in this matrix?

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 4</span>**. Write down the explicit expression for the *log*-determinant of the Jacobian matrix from the previous section.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 5</span>**. Write down the explicit expression for the *log*-determinant of the Jacobian matrix of the *inverse* function $\frac{d \mathbf{x}}{d \mathbf{y}}$.



### Task 2
$$ x_{1:j} = y_{1:j} $$
$$ x_{j+1:d} = (y_{j+1:d}-t_{\theta_2}(y_{1:j})) \odot \exp{(-s_{\theta_1}(y_{1:j}))}$$

### Task 3

\begin{equation*}
J = 
\begin{pmatrix}
I_j & 0 \\
\frac{dy_{j+1:d}}{dx^T_{1:j}} & diag(\exp(s(x_{1:j}))\\
\end{pmatrix}
\end{equation*}

We can notice that this is a lower triangular matrix.

### Task 4

Since the jacobian is a lower triangular matrix the determinant is just the multiplication of the diagonal elements $\exp[\sum_n{s(x_{1:j})_n}] $, so the log-deteminant is simply :
$$\sum_n{s(x_{1:j})_n}$$

### Task 5
According to the inverse function theorem, the matrix inverse of the Jacobian matrix of an invertible function is the Jacobian matrix of the inverse function, so:

$$J_{f^{-1}} = J_{f}^{-1}$$
So the log-determinant would be: 
$$ -\sum_n{s(y_{1:j})_n}$$



In our implementation, we will realize the splitting of variables as masking. The variables to be transformed, $\mathbf{x}_{j+1:d}$, are masked when passing $\mathbf{x}$ to the networks to predict the transformation parameters $\mathbf{s}_{\theta_1}(\mathbf{x}_{1:j})$ and $\mathbf{t}_{\theta_2}(\mathbf{x}_{1:j})$. Also, afterward when applying the transformation (don't forget to exponentiate the scaling!), we mask the parameters for $\mathbf{x}_{1:j}$ so that we have an identity operation for those variables.

For predicting the shifting and scaling parameters for our toy datasets we will be using neural networks with 3 Fully Connected layers with LeakyReLU activations in between. Additionally, for stabilization purposes, we multiply the scaling output $\mathbf{s}_{\theta_1}(\mathbf{x}_{1:j})$ prior to exponentiation with a learnable parameter per dimension `scale_factor` initialized to 0. Meaning, our scaling is initialized to 1 as $\exp(0) = 1$. This prevents sudden large scaling values that can destabilize training (especially in the beginning).

The functions $\mathbf{s}_{\theta_1}(\cdot)$, $\mathbf{t}_{\theta_2}(\cdot)$, and `scale_factor` are already implemented in the provided class `CouplingLayer` below for your convinience.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 6</span>**. Implement missing `forward` and `inverse` methods in the class `CouplingLayer`:
  * The `forward` method should take in `x` and use the mask `self.mask` and the learnable functions `self.s_func`, `self.scale_factor` and `self.t_func` to predict the transformation parameters and compute `y`. This method should also return the parameter `log_det_jac` which is the log-determinant of the Jacobian.
  * The `inverse` method goes in the other direction. It takes in `y` and uses `self.mask`, `self.s_func`, `self.scale_factor` and `self.t_func` to compute `x`. This method should also return the parameter `inv_log_det_jac` which is the log-determinant of the Jacobian of $f^{-1}(\cdot)$.

In [ ]:
class CouplingLayer(nn.Module):
  def __init__(self, mask):
    super(CouplingLayer, self).__init__()

    # mask for splitting (fixed not learnable)
    self.mask = nn.Parameter(mask, requires_grad=False)

    # scaling function and stabilizing scale_factor init. to 0
    self.s_func = nn.Sequential(nn.Linear(in_features=2, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=2))
    self.scale_factor = nn.Parameter(torch.Tensor(2).fill_(0.0))

    # shifting function
    self.t_func = nn.Sequential(nn.Linear(in_features=2, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=2))

  def forward(self, x):
    """
    TODO: replace y and log_det_jac with your code.
    """
    x_part_1 = torch.mul(x, self.mask)
    inv_mask = 1 - self.mask
    x_part_2 = torch.mul(x, inv_mask)
    s = self.s_func(x_part_1)
    t = self.t_func(x_part_1) * inv_mask
    scale_s = torch.mul(s, self.scale_factor)
    exp_scale = torch.exp(scale_s)
    y_part_1 = x_part_1
    y_part_2 = torch.mul(torch.mul(exp_scale, x_part_2) + t, inv_mask)
    y = y_part_1 + y_part_2
    log_det_jac = torch.sum(scale_s*inv_mask,dim=1)
    return y, log_det_jac

  def inverse(self, y):
    """
    TODO: replace x and inv_log_det_jac with your code.
    """
    y_part_1 = torch.mul(y,self.mask)
    inv_mask = 1-self.mask
    y_part_2 = torch.mul(y,inv_mask)
    s = self.s_func(y_part_1)
    t = self.t_func(y_part_1)
    exp_scale = torch.exp(-torch.mul(s,self.scale_factor))
    x_part_2 = torch.mul((y_part_2 - t),exp_scale)
    x_part_1 = y_part_1

    x = torch.concat((x_part_1[:,torch.where(self.mask==1)],x_part_2[:,torch.where(inv_mask==1)]),dim=1)
    inv_log_det_jac = -torch.sum(s)
    return x, inv_log_det_jac

In [ ]:
# test my implementation
mask = torch.tensor([1.,0.])
l = CouplingLayer(mask)
x = torch.tensor([[1.,2.],[3,4]])
y, jac = l(x)
print('y:' ,y, 'J:', jac)
#test inverse:
x_revert, jac_inv = l.inverse(y)
print('x:' ,x_revert, 'J inv :' ,jac_inv)

assert torch.allclose(x,x_revert), "x input and output aren't the same!"



### Coupling Flows

As you might have guessed by now, a coupling layer is powerful yet still limited in its ability to significantly alter the input. This is because it only operates on a chunk of it with element-wise manipulations due to the invertability constraint. We can go on with making our function $f$ more complex. How can we implement more complex invertible functions? The answer is: invertible function composition. We can stack multiple invertible functions $f_1,\dots,f_K$ (e.g. Coupling Layers) after each other, as all together, they still represent a single, invertible function. Specifically, if $\mathbf{y} = f_1(\mathbf{z})$ and $\mathbf{x} = f_2(\mathbf{y})$ are invertible functions, then $\mathbf{x} = f_2 \circ f_1 (\mathbf{z})$ is an invertible function and its inverse is given by $f_1^{-1} \circ f_2^{-1}$. More importantly, the calculation of the log-determinant of the Jacobian in this case is simple using the chain rule.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 7</span>**. Assuming $\mathbf{y} = f_1(\mathbf{z})$ and $\mathbf{x} = f_2(\mathbf{y})$ are coupling layers, calculate the log-determinant of the Jacobian $\frac{d \mathbf{x}}{d \mathbf{z}}$. How is it related to the log-determinant of the Jacobians $\frac{d \mathbf{y}}{d \mathbf{z}}$ and $\frac{d \mathbf{x}}{d \mathbf{y}}$?

### Task 7
$$ \log |J_{f_1,f_2}| = \log(|\frac{dx}{dy}\frac{dy}{dz}|)= \log(|\frac{dx}{dy}||\frac{dy}{dz}|)= \log(|\frac{dx}{dy}|+ \log |\frac{dy}{dz}|) = \sum_n{s(y_{1:j})_n} + \sum_n{s(z_{1:j})_n}$$

Coupling layers generalize to any masking technique we could think of. However, the most common approach is to split the input $\mathbf{x}$ in half using the mask. For our toy 2D datasets comprised of samples $\mathbf{x} = (p_x ,p_y) \in \mathbb{R}^{d=2}$, this means that either $\left\{\mathbf{x}_{1:j}=p_x, \mathbf{x}_{j+1:d}=p_y\right\}$, or $\left\{\mathbf{x}_{1:j}=p_y, \mathbf{x}_{j+1:d}=p_x\right\}$. These correspond to masks $\left[1,0\right]^T$ and $\left[0,1\right]^T$ respectively. 
Note that when we apply multiple coupling layers, we invert the masking every other layer so that each variable is transformed a similar amount of times.

#### Intuition in 1D

Intuitively, using multiple, learnable invertible functions, a normalizing flow attempts to transform $p_z(z)$ slowly into a more complex distribution which should finally be $p_x(x)$. We visualize the idea below
(figure credit - [Lilian Weng](https://lilianweng.github.io/lil-log/2018/10/13/flow-based-deep-generative-models.html)):

<center width="100%"><img src="https://lilianweng.github.io/posts/2018-10-13-flow-models/normalizing-flow.png" width="700px"></center>

Starting from $z_0$, which follows the prior Gaussian distribution, we sequentially apply the invertible functions $f_1,f_2,...,f_K$, until $z_K$ represents $x$.

#### Implementation

Using the `CouplingLayer` class from above, here we will implement a class named `CouplingFlow` that is comprised of stacked coupling layers. This class will have the following attributes:
* `num_layers` - a scalar passed at initialization that will determine the number of coupling layers to stack, refered to later as $K$.
* `self.layers` - a Module list comprised of $K$ stacked coupling layers each with its own mask. Note that the masks are fixed and non-learnable therefore we set their `requires_grad` property to `False` inside `CouplingLayer`.
* `self.prior` - This is the prior/base distribution. Here we will use a standard Gaussian distribution with a unit variance per dimension implemented using the `torch.distributions` module.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 8</span>**. Implement the following 3 methods of this class:
* `log_probability` - method that takes in a batch of samples `x` and returns `log_prob` which is their log-probability $\log p(\mathbf{x})$. For convinience we will assume the overall function $f=f_K\circ f_{K-1} \dots \circ f_1$ satisfies $\mathbf{x} = f(\mathbf{z})$. Hence, to calculate the log-probability you should employ the inverse functions $f_i^{-1}$ starting from the last layer $f_K^{-1}$ (assuming $K$ layers).
* `sample_x` - method that takes in a parameter `num_samples` and returns samples `x` from $p(\mathbf{x})$ alongside their log-probability values `log_prob`. 
* `sample_x_each_step` - method takes in a parameter `num_samples` and returns a list of samples `samples` after each intermediate coupling layer in `self.layers`. The first element of this list is samples from the prior distribution $p(\mathbf{z})$ and the last element is samples from $p(\mathbf{x})$.

In [ ]:
class CouplingFlow(nn.Module):
  def __init__(self, num_layers):
      super(CouplingFlow, self).__init__()

      # concatenate coupling layers with alternating masks
      masks = F.one_hot(torch.tensor([i % 2 for i in range(num_layers)])).float()
      self.layers = nn.ModuleList([CouplingLayer(mask) for mask in masks])

      # define prior distribution to be z~N(0,I)
      self.prior = MultivariateNormal(torch.zeros(2),torch.eye(2))

  def log_probability(self, x):
    """
    TODO: replace log_prob with your code.
    """
    log_det_jac = 0
    num_of_layers = len(self.layers)
    for i, _ in enumerate(self.layers):
        x , log_det_jac_i = self.layers[i](x)
        log_det_jac+=log_det_jac_i
    y=x # switch to y for convinience
    log_p_y= self.prior.log_prob(y)
    log_prob= log_p_y + log_det_jac
    return log_prob

  def sample_x(self, num_samples):
    """
    TODO: replace x and log_prob with your code.
    """
    y = self.prior.sample([num_samples])
    num_of_layers = len(self.layers)
    inv_log_det_jac = 0
    for i, _ in enumerate(self.layers):
        y , inv_log_det_jac_i = self.layers[num_of_layers-i-1].inverse(y)
        inv_log_det_jac+=inv_log_det_jac_i
    x = y
    log_p_y = self.prior.log_prob(y)
    log_prob = log_p_y + inv_log_det_jac # +inv_log_det_jac_i
    return x, log_prob

  def sample_x_each_step(self, num_samples):
    """
    TODO: replace samples with your code.
    """
    log_det_jac = 0
    samples = []
    y = self.prior.sample([num_samples])
    num_of_layers = len(self.layers)
    inv_log_det_jac = 0
    samples.append(y)
    for i, _ in enumerate(self.layers):
        y , inv_log_det_jac_i = self.layers[num_of_layers-i-1].inverse(y)
        samples.append(y)
        #inv_log_det_jac+=inv_log_det_jac_i
        #log_p_y = self.prior.log_prob(y)
        #log_prob = log_p_y + inv_log_det_jac # +inv_log_det_jac_i
    
    return samples

In [ ]:
# Test my imp.
cf = CouplingFlow(2)
x = torch.tensor([[1,2],[3,4]])
lp = cf.log_probability(x)
print('log(p(x)):',lp.data)


In [ ]:
x,log_prob = cf.sample_x(10)
print('Samples from x (2d): \n',x.data, '\nlog prob:' , log_prob.data)

In [ ]:
samples = cf.sample_x_each_step(2)
print([sample.data for sample in samples])

### Training Coupling Flows

Now that we have finished implementing the flow model, we can start training it. Provided below is an already implemented function `train` to train your `CouplingFlow` models. The function recieves 4 arguments:
* `model` - an instance of the class `CouplingFlow`.
* `data` - an instance of the class `ToyDataset`.
* `epochs` - number of epochs to train the model (int). 
* `batch_size` - the batch size to use in training (int).

In [ ]:
# detach tensor and transfer to numpy
def to_np(x):
  return x.detach().numpy()

# Simple training function
def train(model, data, epochs = 100, batch_size = 64):

  # move model into the device
  model = model.to(device)

  # split into training and validation, and create the loaders
  lengths = [int(len(data)*0.9), len(data) - int(len(data)*0.9)]
  train_set, valid_set = random_split(data, lengths)
  train_loader = DataLoader(train_set, batch_size=batch_size)
  valid_loader = DataLoader(valid_set, batch_size=batch_size)

  # define the optimizer and scheduler
  optimizer = Adam(model.parameters(), lr=1e-3)

  # train the model
  train_losses, valid_losses, min_valid_loss = [], [], np.Inf
  with tqdm.tqdm(range(epochs), unit=' Epoch') as tepoch:
    for epoch in tepoch:

      # training loop
      epoch_loss = 0
      model.train(True)
      for batch_index, training_sample in enumerate(train_loader):
          log_prob = model.log_probability(training_sample)
          loss = - log_prob.mean(0)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          epoch_loss += loss
      epoch_loss /= len(train_loader)
      train_losses.append(np.copy(to_np(epoch_loss)))

      # validation loop
      epoch_loss_valid = 0
      model.train(False)
      for batch_index, valid_sample in enumerate(valid_loader):
        log_prob = model.log_probability(valid_sample)
        loss_valid = - log_prob.mean(0)
        epoch_loss_valid += loss_valid
      
      epoch_loss_valid /= len(valid_loader)
      valid_losses.append(np.copy(to_np(epoch_loss_valid)))
      
      # save best model based off validation loss
      if epoch_loss_valid < min_valid_loss:
        model_best = copy.deepcopy(model)
        min_valid_loss = epoch_loss_valid
        epoch_min = epoch

      # report progress with tqdm pbar
      tepoch.set_postfix(train_loss=to_np(epoch_loss), valid_loss=to_np(epoch_loss_valid))
      
  # report best model on val.
  print('\n Best Model achieved {:.4f} validation loss at epoch {} \n'.
        format(min_valid_loss, epoch_min))

  # if the number of samples is too low take the final weights regardless of 
  # valdiation loss due to weak statistics (overfitting avoided by early stopping)
  if lengths[1] < 500:
    model_best = model

  return model_best, train_losses, valid_losses

Here is an example snippet for using this function to train a flow model with $K=4$ layers on a dataset of $N=1500$ samples $\mathbf{x}_i$ from the `'Moons'` distribution for 1000 epochs:

In [ ]:
# seeds to ensure reproducibility
torch.manual_seed(8)
np.random.seed(0)

# dataset
num_samples = 1500
data = ToyDataset('Moons', num_samples=num_samples)

# learning hyper-parameters
K = 4
nepochs = 1000

# instantiate model and optimize the parameters
Flow_model = CouplingFlow(num_layers=K)
moon_model, train_loss, valid_loss = train(Flow_model, data, epochs=nepochs)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 9</span>**. For each of the 4 provided distributions (use the default value of `num_gaussians=5` for `'GaussiansRot'`), use a similar snippet to repeat the following:
  * Create a `ToyDataset` instance with $N=1500$ samples from the distribution.
  * Learn a `CouplingFlow` model with $K=4$ layers, by training for 500 epochs. For the `'Moons'` dataset train for 1000 epochs.
  * Plot the estimated density $p(\mathbf{x})$ in $\mathbb{R}^2$. To achieve this, use the method `log_probability` to calculate the log-probability $\log p(\mathbf{x})$ at a pre-determined grid of points $\mathbf{x} \in \left[x_{\min},x_{\max}\right]\times\left[y_{\min},y_{\max}\right]\subset\mathbb{R}^2$ (e.g. using `np.meshgrid`). Sample each coordinate with at least 100 points (i.e. a grid of $100 \times 100$ positions in 2D). Plot the resulting 2D distribution $p(\mathbf{x})=\exp (\log p(\mathbf{x}))$ as an image where the value in each pixel is $p(\mathbf{x})$.
  * Plot samples from intermediate flow layers, including the prior $p(\mathbf{z})$ and the modelled $p(\mathbf{x})$. To achieve this, use the method `sample_x_each_layer` with $N=1000$ samples. Plot the resulting samples from each layer in the **same** axis limits to visualize the transformation of each coupling layer separately. You can use `plt.subplot(..., sharex=True, sharey=True)` to achieve link the axis of all subplots.


Implementation tips:
* Use the same seeds as the example snippet for reproducibility. This will also ensure a non-diverging erroneous behavior with other seeds.
* To make sure the model is not overfitting you can look at the training and the valdiation loss outputs of the provided function `train`. Do not include these in your report, use them just for sanity check.
* Estimate the log-probability in a reasonable vicinity of the training domain. For far away coordinates from the training data, the estimation could be poor locally and might bias the dynamic range of your plot.
* To avoid code duplication, you are encouraged to implement two plotting functions: one for plotting the density, and one for plotting the transformations across layers.

In [ ]:
def plot_model_dist_on_grid(title,model,xmin,xmax,ymin,ymax):
        g = torch.meshgrid(torch.linspace(xmin,xmax,100),torch.linspace(ymin,ymax,100))
        grid = torch.concat((g[0],g[1]),dim=0).reshape(2, -1).T
        log_p = model.log_probability(grid)
        log_p_mesh = (log_p.T).reshape(g[0].shape[0],g[0].shape[1])
        log_p_mesh_np = log_p_mesh.detach().numpy()
        p = np.exp(log_p_mesh_np)
        plt.figure(figsize = (5,5))
        plt.imshow(p,extent =[xmin,
                             xmax,
                             ymin,
                             ymax])
        plt.title(title)
        plt.savefig(title +'.png')

def train_dist(dist, N=1500,nepochs=500,K=4,num_gaussians=5):
    torch.manual_seed(8)
    np.random.seed(0)
    num_samples = N
    data = ToyDataset(dist, num_samples=num_samples, num_gaussians=num_gaussians)
    # instantiate model and optimize the parameters\
    Flow_model = CouplingFlow(num_layers=K)
    model, train_loss, valid_loss = train(Flow_model, data, epochs=nepochs)
    return model,train_loss,valid_loss




In [ ]:
circles_model, train_loss, valid_loss = train_dist('Circles',nepochs=500)

In [ ]:
plot_model_dist_on_grid('Circles',circles_model,-1,1,-1,1)

In [ ]:
GaussiansGrid_model, train_loss, valid_loss = train_dist('GaussiansGrid',nepochs=500)

In [ ]:
plot_model_dist_on_grid('GaussiansGrid',GaussiansGrid_model,-2,4,-2,4)

In [ ]:
GaussiansRot_model, train_loss, valid_loss = train_dist('GaussiansRot',nepochs=500)

In [ ]:
plot_model_dist_on_grid('GaussiansRot',GaussiansRot_model,-4,4,-4,4)

In [ ]:
Moon_model ,train_loss, valid_loss = train_dist('Moons',nepochs=1000)

In [ ]:
plot_model_dist_on_grid('Moons',Moon_model,-1,2,-1,2)

In [ ]:
def plot_intermidiate_dists(title,model,num_samples,k=4):
    samples = model.sample_x_each_step(num_samples)
    titles = ['$p(z_0)$','$p(z_1)$','$p(z_2)$','$p(z_3)$','$p(x)$']
    numofaxes=5
    if (k==2):
        titles = ['$p(z_0)$','$p(z_1)$','$p(x)$']
        numofaxes=3
    fig,axes = plt.subplots(1,numofaxes,sharex=True,sharey=True,figsize=[20,4])
    fig.suptitle(title)
    for i, (sample, ax) in enumerate(zip(samples,axes)):
        sample = torch.detach(sample).numpy()
        ax.scatter(sample[:,0],sample[:,1],s=5,alpha=0.3)
        ax.set_title(titles[i])
    plt.gcf().set_dpi(300)
    plt.savefig(title +'.png')

dists = ['Circles', 'Moons', 'GaussiansGrid', 'GaussiansRot']
models = [circles_model, Moon_model,GaussiansGrid_model,GaussiansRot_model]
num_samples=1000
for model,dist in zip(models,dists):
    title = dist + ' samples per layer'
    plot_intermidiate_dists(title,model,num_samples)

#### Analyzing Coupling Layers

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 10</span>**. Train two flow models with a varying number of coupling layers $K=\left\{2, 4\right\}$ for 250 epochs, using $N=1500$ samples from the `'GaussiansRot'` dataset with `num_gaussians=5`. Compare the resulting estimated density $p(\mathbf{x})$ (using a grid of $100 \times 100$ points) and the intermediate distributions of $N=1000$ samples throughout the coupling layers of the model. Which model fits $p(\mathbf{x})$ better? What do you conclude regarding the effect of model depth? explain the result in your report and attach the resulting plots.   

In [ ]:
dist = 'GaussiansRot'
k=2
nepochs=250
N=1500
model_k2 ,train_loss, valid_loss = train_dist(dist,nepochs=nepochs,K=k,N=N)
title = dist + ' 2 Layers'
plot_model_dist_on_grid(title,model_k2,-4,4,-4,4)
plot_intermidiate_dists(title +' samples per layer',model_k2,num_samples=1000,k=2)

In [ ]:
k=4
nepochs=250
N=1500
model_k4 ,train_loss, valid_loss = train_dist(dist,nepochs=nepochs,K=k,N=N)
title = dist + ' 4 Layers'
plot_model_dist_on_grid(title,model_k4,-4,4,-4,4)
plot_intermidiate_dists(title+' samples per layer',model_k4,num_samples=1000)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 11</span>**. Train two flow models with $K=4$ layers for 400 epochs, using a varying number of $N=\left\{1500, 3000\right\}$ samples from the `'GaussiansRot'` dataset with `num_gaussians=5`. Compare the resulting estimated density $p(\mathbf{x})$ (using a grid of $100 \times 100$ points) and the intermediate distributions of $N=1000$ samples throughout the coupling layers of the model. Which model fits $p(\mathbf{x})$ better? What do you conclude regarding the effect of training set size? explain the result in your report and attach the resulting plots.

In [ ]:
dist = 'GaussiansRot'
n=1500
nepochs=400
K=4
model_n1500 ,train_loss, valid_loss = train_dist(dist,nepochs=nepochs,K=K,N=n)
title = dist + ' 1500 samples'
plot_model_dist_on_grid(title,model_n1500,-4,4,-4,4)
plot_intermidiate_dists(title+' samples per layer',model_n1500,num_samples=1000)

In [ ]:
dist = 'GaussiansRot'
n=3000
K=4
nepochs= 400
model_n3000 ,train_loss, valid_loss = train_dist(dist,nepochs=nepochs,K=K,N=n)
title = dist + ' 3000 samples'
plot_model_dist_on_grid(title,model_n3000,-4,4,-4,4)
plot_intermidiate_dists(title+' samples per layer',model_n3000,num_samples=1000)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 12</span>**. Train two flow models with $K=4$ layers for 250 epochs, using $N=1500$ samples from the `'GaussiansRot'` dataset with a varying number of Gaussians `num_gaussians`$=\left\{3, 7\right\}$. Compare the resulting estimated density $p(\mathbf{x})$ (using a grid of $100 \times 100$ points) and the intermediate distributions of $N=1000$ samples throughout the coupling layers of the model. Which distribution $p(\mathbf{x})$ is fitted better by the model? What do you conclude regarding the effect of data complexity? explain the result in your report and attach the resulting plots.

In [ ]:
dist = 'GaussiansRot'
N=1500
K=4
num_gaussians=3
nepochs=250
model_3gaussians ,train_loss, valid_loss = train_dist(dist,nepochs=nepochs,K=K,N=N,num_gaussians=num_gaussians)
title = dist + ' 3 gaussians'
plot_model_dist_on_grid(title,model_3gaussians,-4,4,-4,4)
plot_intermidiate_dists(title+' samples per layer',model_3gaussians,num_samples=1000)

In [ ]:
dist = 'GaussiansRot'
num_gaussians=7
model_7gaussians ,train_loss, valid_loss = train_dist(dist,nepochs=nepochs,K=K,N=N,num_gaussians=num_gaussians)
title = dist + ' 7 gaussians'
plot_model_dist_on_grid(title,model_7gaussians,-4,4,-4,4)
plot_intermidiate_dists(title+' samples per layer',model_7gaussians,num_samples=1000)

### Conclusion

In conclusion, we have seen how to implement our own normalizing flow on toy 2D datasets. However, as mentioned in the beginning of Part II, similar models with significantly more layers and additional tricks can be used to model images (e.g. [Glow](http://papers.nips.cc/paper/8224-glow-generative-flow-with-invertible-1x1-convolutions.pdf)). The most common flow element, the coupling layer, is simple to implement, and yet effective. Normalizing flows are an interesting generative model compared to GANs, as they allow an exact likelihood estimate in continuous space, and we have the guarantee that every possible input $x$ has a corresponding latent vector $z$. Recent advances in [Neural ODEs](https://arxiv.org/pdf/1806.07366.pdf) allow a flow with infinite number of layers, called Continuous Normalizing Flows, whose potential is yet to fully explore.

## References and Credits

[1] Dinh, L., Sohl-Dickstein, J., and Bengio, S. (2017). “Density estimation using Real NVP,” In: 5th International Conference on Learning Representations, ICLR 2017. [Link](https://arxiv.org/abs/1605.08803)

[2] Kingma, D. P., and Dhariwal, P. (2018). “Glow: Generative Flow with Invertible 1x1 Convolutions,” In: Advances in Neural Information Processing Systems, vol. 31, pp. 10215--10224. [Link](http://papers.nips.cc/paper/8224-glow-generative-flow-with-invertible-1x1-convolutions.pdf)

[3] University of Amsterdam, Deep Learning 1, Tutorial 11. [Link](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial11/NF_image_modeling.html#Normalizing-Flows-as-generative-model)

[4] Technical University of Munich, Machine Learning for Graphs and Sequential Data, Generative Models. [Link](https://www.in.tum.de/en/daml/teaching/summer-term-2020/machine-learning-for-graphs-and-sequential-data/)